In [1]:
# data manipulation 
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
# modeling utilities
from sklearn import metrics
from sklearn import preprocessing
from sklearn import  linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import OrdinalEncoder


In [2]:
credit_db = pd.read_csv('clean_dataset.csv')

In [3]:
credit_db.head()

,Gender,Age,Debt,Married,BankCustomer,Industry,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,1,30.83,0.000,1,1,Industrials,White,1.25,1,1,1,0,ByBirth,202,0,1
1,0,58.67,4.460,1,1,Materials,Black,3.04,1,1,6,0,ByBirth,43,560,1
2,0,24.50,0.500,1,1,Materials,Black,1.50,1,0,0,0,ByBirth,280,824,1
3,1,27.83,1.540,1,1,Industrials,White,3.75,1,1,5,1,ByBirth,100,3,1
4,1,20.17,5.625,1,1,Industrials,White,1.71,1,0,0,0,ByOtherMeans,120,0,1


In [4]:
credit_db

,Gender,Age,Debt,Married,BankCustomer,Industry,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,1,30.83,0.000,1,1,Industrials,White,1.25,1,1,1,0,ByBirth,202,0,1
1,0,58.67,4.460,1,1,Materials,Black,3.04,1,1,6,0,ByBirth,43,560,1
2,0,24.50,0.500,1,1,Materials,Black,1.50,1,0,0,0,ByBirth,280,824,1
3,1,27.83,1.540,1,1,Industrials,White,3.75,1,1,5,1,ByBirth,100,3,1
4,1,20.17,5.625,1,1,Industrials,White,1.71,1,0,0,0,ByOtherMeans,120,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,1,21.08,10.085,0,0,Education,Black,1.25,0,0,0,0,ByBirth,260,0,0
686,0,22.67,0.750,1,1,Energy,White,2.00,0,1,2,1,ByBirth,200,394,0
687,0,25.25,13.500,0,0,Healthcare,Latino,2.00,0,1,1,1,ByBirth,200,1,0
688,1,17.92,0.205,1,1,ConsumerStaples,White,0.04,0,0,0,0,ByBirth,280,750,0


In [5]:
encoder = OrdinalEncoder()
credit_db["Industry"] = encoder.fit_transform(credit_db[["Industry"]])
credit_db["Ethnicity"] = encoder.fit_transform(credit_db[["Ethnicity"]])
credit_db["Citizen"] = encoder.fit_transform(credit_db[["Citizen"]])

In [6]:
credit_db

,Gender,Age,Debt,Married,BankCustomer,Industry,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,1,30.83,0.000,1,1,7.0,4.0,1.25,1,1,1,0,0.0,202,0,1
1,0,58.67,4.460,1,1,9.0,1.0,3.04,1,1,6,0,0.0,43,560,1
2,0,24.50,0.500,1,1,9.0,1.0,1.50,1,0,0,0,0.0,280,824,1
3,1,27.83,1.540,1,1,7.0,4.0,3.75,1,1,5,1,0.0,100,3,1
4,1,20.17,5.625,1,1,7.0,4.0,1.71,1,0,0,0,1.0,120,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,1,21.08,10.085,0,0,3.0,1.0,1.25,0,0,0,0,0.0,260,0,0
686,0,22.67,0.750,1,1,4.0,4.0,2.00,0,1,2,1,0.0,200,394,0
687,0,25.25,13.500,0,0,6.0,2.0,2.00,0,1,1,1,0.0,200,1,0
688,1,17.92,0.205,1,1,2.0,4.0,0.04,0,0,0,0,0.0,280,750,0


In [7]:
X, X_test, y, y_test = train_test_split(credit_db.iloc[:,0:-1], credit_db.iloc[:,-1], 
                                                    test_size=0.33, random_state=42)

In [8]:
model = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [9]:
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.823 (0.064)
